### Melon 100 Chart
* 100곡의 노래의 제목, Song ID추출 (정규표현식 사용)
* Song ID로 상세페이지를 100번 요청해서 노래 상세정보 추출
* 100곡의 상세정보를 json file에 저장한다.
* json file을 Pandas의 DataFrame 객체에 저장해서 표데이터를 만들기
* 표 데이터를 MariaDB에 저장하기

#### 1. 100곡의 노래 제목, Song ID추출 (정규표현식 사용)

In [3]:
import requests
from bs4 import BeautifulSoup
import re

url = "https://www.melon.com/chart/index.htm"
req_header = {
    'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}

# 요청을 보내기
res = requests.get(url, headers=req_header)
print(res.status_code)
# 응답결과가 정상이면
if res.ok:
    # 응답 객체에 포함된 test 추출
    html = res.text
    # 추출한 text를 파싱하기 위한 beautifulSoup 객체 생성
    soup = BeautifulSoup(html, 'html.parser')
    # <div id='tb_list'> div 태그 선택
    print(len(soup.select('div#tb_list')))
    # <div id='tb_list'> 하위에 있는 태그 선택, 101개
    print(len(soup.select('div#tb_list tr')))
    # <div id='tb_list'> 하위에 있는 tr 하위에 <div class='wrap_song_info'>
    print(len(soup.select('div#tb_list tr .wrap_song_info')))
    print(len(soup.select('div.wrap_song_info')))
    # <div id='id_list'> 하위에 있는 tr 하위에 <div class ='wrap_song_info'>
    # 하위에 <a hrep='playSong'>
    print(len(soup.select('div#tb_list tr .wrap_song_info a[href*=\'playSong\']')))
    
    atag_list = soup.select("div#tb_list tr .wrap_song_info a[href*='playSong']")
    print('----------------------------------------')
    print(type(atag_list), atag_list[2].text, type(atag_list[2].text))
    print('')
        
    # 100곡 노래 저장할 list
    song_list = []
    for idx, atag in enumerate(atag_list, 1):
        # 1곡 노래 저장할 dict
        song_dict = {}
        
        # a tag 의 content
        title = atag.text
        # a tag 의 속성 값
        href = atag['href']
        
        # 정규표현식의 패턴으로 숫자를 검색
        matched = re.search(r'(\d+)\);', href)
        # print(matched, matched.group(0), matched.group(1)) # group(1) 값을 주어 숫자만 가져옴
        song_id = matched.group(1)
        print(idx, title, song_id)
        song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        
        # 추출한 song title과 url을 song_dict에 저장
        song_dict['title'] = title
        song_dict['url'] = song_url
        # 1개의 노래정보를 담고 있는 song_dict를 song_list에 추가
        song_list.append(song_dict)
        
    print(len(song_list))
    print(song_list[0:3])

200
1
101
200
200
100
----------------------------------------
<class 'bs4.element.ResultSet'> Dynamite <class 'str'>

1 VVS (Feat. JUSTHIS) (Prod. GroovyRoom) 33077590
2 밤하늘의 별을(2020) 33061995
3 Dynamite 32872978
4 잠이 오질 않네요 33013877
5 내일이 오면 (Feat. 기리보이, BIG Naughty (서동현)) 33115807
6 힘든 건 사랑이 아니다 32998018
7 Lovesick Girls 32961718
8 취기를 빌려 (취향저격 그녀 X 산들) 32794652
9 Life Goes On 33077234
10 오래된 노래 3894276
11 CREDIT (Feat. 염따, 기리보이, Zion.T) 33150997
12 Savage Love (Laxed - Siren Beat) (BTS Remix) 32962258
13 내 마음이 움찔했던 순간 (취향저격 그녀 X 규현) 32871975
14 나랑 같이 걸을래 (바른연애 길잡이 X 적재) 33011180
15 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 32061975
16 Achoo (Feat. pH-1, HAON) (Prod. GroovyRoom) 33115804
17 에잇(Prod.&Feat. SUGA of BTS) 32578498
18 ON AIR (Feat. 로꼬, 박재범 & GRAY) 33150993
19 When We Disco (Duet with 선미) 32853712
20 Freak (Prod. Slom) 33077591
21 혼술하고 싶은 밤 33104090
22 흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야 32003395
23 늦은 밤 너의 집 앞 골목길에서 32156286
24 어떻게 지내 (Prod. By VAN.C) 32438894
25 How You Like That 32720013
26 딩가딩가

## Song ID로 상세페이지를 100번 요청해서 노래 상세정보 추출

In [4]:
import requests
from bs4 import BeautifulSoup
import re

req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
}
# 노래 100곡의 상세정보를 저장할 dict
song_detail_list = []

for song in song_list:
    # 노래 1곡의 상세정보를 저장할 dict
    
    song_detail_dict = {}
    # print(song['title'], song['link'])
    song_url = song['url']
    res = requests.get(song_url, headers=req_header)
    print(res.status_code)
    if res.ok:
        html = res.text
        soup = BeautifulSoup(html,'html.parser')
        #노래제목
        song_detail_dict['곡명'] = song['title']
        #가수
        song_detail_dict['가수'] = soup.select('a[href*="goArtistDetail"] span')[0].text
        #앨범
        song_detail_dict['앨범'] = soup.select('div.meta dd')[0].text
        #발매일
        song_detail_dict['발매일'] = soup.select('div.meta dd')[1].text
        #장르
        song_detail_dict['장르'] = soup.select('div.meta dd')[2].text
        #가사
        lyric = soup.select('div#d_video_summary')[0].text
        regexp = re.compile(r'[\n\r\t]')        
        # Pattern과 매칭되는 문자열로 empty string으로 치환
        song_detail_dict['가사'] = regexp.sub('',lyric.strip())
        #각각의 Song정보를 담고 있는 dict를 list에 저장
        song_detail_list.append(song_detail_dict)
#         print('-----')
#         print(song_detail_dict)    

print(len(song_detail_list))
print(song_detail_list[0:3])

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
100
[{'곡명': 'VVS (Feat. JUSTHIS) (Prod. GroovyRoom)', '가수': '미란이 (Mirani)', '앨범': '쇼미더머니 9 Episode 1', '발매일': '2020.11.21', '장르': '랩/힙합', '가사': "보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아끼워 다 새 거My life's shining like a VVS VVS내 삶은 빛나 VVS VVS보란 듯 나와 TV에Now you see me babe내 삶은 빛나 VVS VVSSkrt-skrtHey new water new vv난 알바 째고 무대 위Yeah go get it go get it가사 위 가난이 빛나지안 가 무한리필살아봐야겠어 내 빌딩 Yeah개 같던 세상의뒤통수 치러 왔지더 크게 Callin’ ma name모두 날 보고 놀래미란이가 TV에떼버려 Tag사 새롭게 Yeah yeah타고 비행Skrr skrr 난 올라가Skrr skrr 난 빛이 나내가 뭐라 했어 Mom꺼내겠다고 포차맨 밑바닥의 소녀엄마의 술병이 날 만들어허기져 이를 꽉 물어Chit chat bout me 덤벼 겁쟁이 너VVS on ma neck 꿈

### 100곡의 상세정보를 json file에 저장한다.

In [5]:
import json

with open ('data/songs.json', 'w', encoding='utf-8') as file:
    json.dump(song_detail_list, file)

with open('data/songs.json', 'r', encoding='utf-8') as f:
    songs_json = json.load(f)

### json file을 Pandas의 DataFrame 객체에 저장해서 표데이터를 만들기

In [9]:
import pandas as pd

# DataFrame 객체 생성
song_df = pd.DataFrame(columns=['곡명', '가수','앨범', '발매일', '장르', '가사'])
# Song_df

for song in songs_json:
    # dict객체를 Series 객체에 저장
    series_obj = pd.Series(song)
    # Series 객체를 DataFrame 객체에 append 합니다.
    song_df = song_df.append(series_obj, ignore_index=True)  
    
song_df.head()



,곡명,가수,앨범,발매일,장르,가사
0,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,2020.11.21,랩/힙합,보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아...
1,밤하늘의 별을(2020),경서,밤하늘의 별을(2020),2020.11.14,발라드,밤하늘의 별을 따서 너에게 줄래 너는 내가 사랑하니까 더 소중하니까 오직 너 아니면...
2,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,Cos ah ahI’m in the stars tonightSo watch me b...
3,잠이 오질 않네요,장범준,잠이 오질 않네요,2020.10.24,발라드,당신은 날 설레게 만들어조용한 내 마음 자꾸만 춤추게 해얼마나 얼마나 날 떨리게 하...
4,"내일이 오면 (Feat. 기리보이, BIG Naughty (서동현))",릴보이 (lIlBOI),쇼미더머니 9 Episode 3,2020.12.05,랩/힙합,See I just wannashare the vibe with ya우리가 10년 ...


In [7]:
song_df.tail()

,곡명,가수,앨범,발매일,장르,가사
95,Blue & Grey,방탄소년단,BE,2020.11.20,발라드,Where is my angel하루의 끝을 드리운Someone come and sa...
96,12월의 어느 겨울…,윤도 (YoonDo),12월의 어느 겨울…,2020.12.26,발라드,십이월의 어느 겨울밤하얀 첫눈이 올 때쯤내려오는 눈꽃을 보며걷고 싶어 너와 함께온통...
97,내 방을 여행하는 법,방탄소년단,BE,2020.11.20,R&B/Soul,떠나볼까 let me fly to my시선을 낮추고 어디든 막 zoom지금 나와 l...
98,요즘,양다일,요즘,2020.12.29,발라드,해가 질 때쯤에눈을 뜨고 나면다가오는 어둠 속에나만 홀로 깨어 있는 듯해고요한 거리...
99,크리스마스니까,성시경,Jelly Christmas 2012 HEART PROJECT,2012.12.06,발라드,오늘은 뭔가 좀 다른 기분 Feeling 이젠 때가 된 것 같은 느낌 Love U ...


In [8]:
song_df.sample(3)

,곡명,가수,앨범,발매일,장르,가사
53,가을밤에 든 생각,잔나비,잔나비 소곡집 l,2020.11.06,"인디음악, 록/메탈",머나먼 별빛 저 별에서도노랠 부르는 사랑 살겠지밤이면 오손도손 그리운 것들 모아서노...
11,Savage Love (Laxed - Siren Beat) (BTS Remix),Jawsh 685,Savage Love (Laxed - Siren Beat) [BTS Remix],2020.10.02,POP,Savage loveDid somebody did somebodyBreak your...
23,어떻게 지내 (Prod. By VAN.C),오반 (OVAN),어떻게 지내,2020.03.05,"랩/힙합, 인디음악",어떻게 지내 식상한 말이내 가사가 될 줄 몰랐어얼마 안됐네 지나온 날이다 알아도 그...


In [10]:
print('shape : ', song_df.shape)
print('columns : ', song_df.columns)
print('index : ',song_df.index)
print('values : ', song_df.values[0:1]) # 2차원 배열

shape :  (100, 6)
columns :  Index(['곡명', '가수', '앨범', '발매일', '장르', '가사'], dtype='object')
index :  RangeIndex(start=0, stop=100, step=1)
values :  [['VVS (Feat. JUSTHIS) (Prod. GroovyRoom)' '미란이 (Mirani)'
  '쇼미더머니 9 Episode 1' '2020.11.21' '랩/힙합'
  "보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아끼워 다 새 거My life's shining like a VVS VVS내 삶은 빛나 VVS VVS보란 듯 나와 TV에Now you see me babe내 삶은 빛나 VVS VVSSkrt-skrtHey new water new vv난 알바 째고 무대 위Yeah go get it go get it가사 위 가난이 빛나지안 가 무한리필살아봐야겠어 내 빌딩 Yeah개 같던 세상의뒤통수 치러 왔지더 크게 Callin’ ma name모두 날 보고 놀래미란이가 TV에떼버려 Tag사 새롭게 Yeah yeah타고 비행Skrr skrr 난 올라가Skrr skrr 난 빛이 나내가 뭐라 했어 Mom꺼내겠다고 포차맨 밑바닥의 소녀엄마의 술병이 날 만들어허기져 이를 꽉 물어Chit chat bout me 덤벼 겁쟁이 너VVS on ma neck 꿈 앞에녹슨 팔찌 버려 문 앞에구제 벨트 아직 허리에원망하던 과거와 춤출래인디고 카니발 Skrt-skrt목숨은 다 꺼내놓고자판기 밑 먼지 덕지덕지 붙은동전까지 털어-털어여기까지 왔지What you know about me내 속 이야기 털어놓으려지갑 털어 차린 술상 위넌 폰만 바라봤잖아이제 그 스크린에서 날 봐라 I'ma먹고서 Fly 너가 나갈 건 삔또지악플들이 돈과 함께 쏟아지니Got a nice ring to it꼬우면 네 오빠인지 아저씨인지한테일러바치고 한 판 더 뜨자 해 빛이나는 Solo라니까 뭘 그렇게 재니다 드루와 

In [12]:
song_df.info

<bound method DataFrame.info of                                         곡명            가수  \
0   VVS (Feat. JUSTHIS) (Prod. GroovyRoom)  미란이 (Mirani)   
1                            밤하늘의 별을(2020)            경서   
2                                 Dynamite         방탄소년단   
3                                잠이 오질 않네요           장범준   
4   내일이 오면 (Feat. 기리보이, BIG Naughty (서동현))  릴보이 (lIlBOI)   
..                                     ...           ...   
95                             Blue & Grey         방탄소년단   
96                             12월의 어느 겨울…   윤도 (YoonDo)   
97                             내 방을 여행하는 법         방탄소년단   
98                                      요즘           양다일   
99                                 크리스마스니까           성시경   

                                    앨범         발매일        장르  \
0                    쇼미더머니 9 Episode 1  2020.11.21      랩/힙합   
1                        밤하늘의 별을(2020)  2020.11.14       발라드   
2           Dynamite (DayTime Version)  2020.08.24     

In [13]:
# 1개의 열(컬럼) 선택
print(type(song_df['가수']))
song_df['가수']

<class 'pandas.core.series.Series'>


0     미란이 (Mirani)
1               경서
2            방탄소년단
3              장범준
4     릴보이 (lIlBOI)
          ...     
95           방탄소년단
96     윤도 (YoonDo)
97           방탄소년단
98             양다일
99             성시경
Name: 가수, Length: 100, dtype: object

In [14]:
# 가수별로 Row Counting
song_df['가수'].value_counts()

방탄소년단               7
아이유                 5
릴보이 (lIlBOI)        4
폴킴                  4
머쉬베놈 (MUSHVENOM)    3
                   ..
경서예지                1
IZ*ONE (아이즈원)       1
임영웅                 1
성시경                 1
잔나비                 1
Name: 가수, Length: 73, dtype: int64

In [15]:
# 장르별로 Row Counting
song_df['장르'].value_counts()

발라드               27
랩/힙합              19
댄스                16
POP               11
R&B/Soul           6
발라드, 국내드라마         5
발라드, 인디음악          4
록/메탈               3
랩/힙합, 인디음악         2
R&B/Soul, 인디음악     2
록/메탈, 국내드라마        2
성인가요               1
인디음악, 록/메탈         1
인디음악, 포크/블루스       1
Name: 장르, dtype: int64

In [16]:
# unique한 장르 가져오기
song_df['장르'].unique()

array(['랩/힙합', '발라드', '댄스', '인디음악, 포크/블루스', 'POP', '발라드, 인디음악', '록/메탈',
       '록/메탈, 국내드라마', '랩/힙합, 인디음악', '발라드, 국내드라마', 'R&B/Soul, 인디음악',
       'R&B/Soul', '인디음악, 록/메탈', '성인가요'], dtype=object)

In [17]:
# unique한 가수 가져오기
song_df['가수'].unique()

array(['미란이 (Mirani)', '경서', '방탄소년단', '장범준', '릴보이 (lIlBOI)', '임창정',
       'BLACKPINK', '산들', '스탠딩 에그', 'Jawsh 685', '규현 (KYUHYUN)', '적재',
       'AKMU (악동뮤지션)', '아이유', '박진영', '벤', '노을', '오반 (OVAN)',
       '마마무 (Mamamoo)', 'Khundi Panda', '폴킴', '조정석', '창모 (CHANGMO)',
       '오마이걸 (OH MY GIRL)', '순순희', '환불원정대', '스윙스', '경서예지', 'TWICE (트와이스)',
       '마크툽 (MAKTUB)', '백지영', '화사 (Hwa Sa)', '이승기', 'Maroon 5',
       '가호 (Gaho)', '전상근', '이예준', '머쉬베놈 (MUSHVENOM)', '이하이', '신예영',
       '제시 (Jessi)', 'Sia', '잔나비', 'Tones And I', '태연 (TAEYEON)',
       'Mariah Carey', '황인욱', '블루 (BLOO)', 'Anne-Marie', 'Ariana Grande',
       '먼데이 키즈 (Monday Kiz)', '비', 'IZ*ONE (아이즈원)', '정준일',
       '엠씨더맥스 (M.C the MAX)', '송하예', '전미도', '임영웅', '#안녕', 'Dua Lipa',
       '허각', '지코 (ZICO)', '싹쓰리 (유두래곤, 린다G, 비룡)', '청하', '원슈타인', 'Etham',
       'Conan Gray', 'Crush', 'Lauv', 'aespa', '윤도 (YoonDo)', '양다일',
       '성시경'], dtype=object)

### 특정 행과 열을 선택
* loc[] , iloc[] 사용
 - Slicing 사용해서 구간을 선택
 - 특정행과 특정열을 선택
 - 조건식을 만족하는 행과 열을 선택

In [21]:
# loc[] - location의 약자, 특정행과 열을 선택할 때 사용
# loc[ 행을 선택하는 기준 , 열을 선택하는 기준]

# 인덱스과 0 부터 5까지인 행과 모든 열을 선택 - Slicing
song_df.loc[0:5, :]


,곡명,가수,앨범,발매일,장르,가사
0,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,2020.11.21,랩/힙합,보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아...
1,밤하늘의 별을(2020),경서,밤하늘의 별을(2020),2020.11.14,발라드,밤하늘의 별을 따서 너에게 줄래 너는 내가 사랑하니까 더 소중하니까 오직 너 아니면...
2,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,Cos ah ahI’m in the stars tonightSo watch me b...
3,잠이 오질 않네요,장범준,잠이 오질 않네요,2020.10.24,발라드,당신은 날 설레게 만들어조용한 내 마음 자꾸만 춤추게 해얼마나 얼마나 날 떨리게 하...
4,"내일이 오면 (Feat. 기리보이, BIG Naughty (서동현))",릴보이 (lIlBOI),쇼미더머니 9 Episode 3,2020.12.05,랩/힙합,See I just wannashare the vibe with ya우리가 10년 ...
5,힘든 건 사랑이 아니다,임창정,힘든 건 사랑이 아니다,2020.10.19,발라드,내가 널 떠났어야 했는데 왜 떠나야 하는지도 아는데 어떤 아무 말도 아무것도 줄 게...


In [23]:
# Slicing으로 인덱스가 0부터 8까지 1줄 씩 skip, 모든 열
song_df.loc[0:8:2, :]

,곡명,가수,앨범,발매일,장르,가사
0,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,2020.11.21,랩/힙합,보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아...
2,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,Cos ah ahI’m in the stars tonightSo watch me b...
4,"내일이 오면 (Feat. 기리보이, BIG Naughty (서동현))",릴보이 (lIlBOI),쇼미더머니 9 Episode 3,2020.12.05,랩/힙합,See I just wannashare the vibe with ya우리가 10년 ...
6,Lovesick Girls,BLACKPINK,THE ALBUM,2020.10.02,댄스,영원한 밤창문 없는 방에 우릴 가둔 loveWhat can we say매번 아파도 ...
8,Life Goes On,방탄소년단,BE,2020.11.20,랩/힙합,어느 날 세상이 멈췄어아무런 예고도 하나 없이봄은 기다림을 몰라서눈치 없이 와버렸어...


In [27]:
# Slicing으로 인덱스가 0부터 8까지 1줄 씩 skip, '곡명'부터 '앨범'까지의 열을 선택
song_df.loc[1:8:2, '곡명':'앨범']

,곡명,가수,앨범
1,밤하늘의 별을(2020),경서,밤하늘의 별을(2020)
3,잠이 오질 않네요,장범준,잠이 오질 않네요
5,힘든 건 사랑이 아니다,임창정,힘든 건 사랑이 아니다
7,취기를 빌려 (취향저격 그녀 X 산들),산들,취기를 빌려 (취향저격 그녀 X 산들)


In [38]:
# 특정행(4, 10, 20)과 모든 열을 선택 (열은 생략 가능)
song_df.loc[[4, 10, 20], :]

,곡명,가수,앨범,발매일,장르,가사
4,"내일이 오면 (Feat. 기리보이, BIG Naughty (서동현))",릴보이 (lIlBOI),쇼미더머니 9 Episode 3,2020.12.05,랩/힙합,See I just wannashare the vibe with ya우리가 10년 ...
10,"CREDIT (Feat. 염따, 기리보이, Zion.T)",릴보이 (lIlBOI),쇼미더머니 9 Final,2020.12.19,랩/힙합,요즘 들어 바쁘게 시간을 보냈지아끼던 내 신발 색이 바랬으니불과 몇 달 만에많은 것...
20,혼술하고 싶은 밤,벤,혼술하고 싶은 밤,2020.12.02,발라드,위로받고 싶은 날 내겐 말해도 돼위로받고 싶은 날 내겐 기대도 돼비에 섞인 음악 소...


In [39]:
# 특정행(10,30,60)과 가수, 앨범, 발매일
song_df.loc[[10,30,60], ['가수', '앨범', '발매일']]

,가수,앨범,발매일
10,릴보이 (lIlBOI),쇼미더머니 9 Final,2020.12.19
30,아이유,Love poem,2019.11.18
60,Anne-Marie,Speak Your Mind (Deluxe),2018.08.03


In [47]:
# 조건식을 만족하는 행과 열을 선택
# 가수가 방탄소년단
print(type(song_df['가수'] == '방탄소년단'))
song_df['가수'] == '방탄소년단'

<class 'pandas.core.series.Series'>


0     False
1     False
2      True
3     False
4     False
      ...  
95     True
96    False
97     True
98    False
99    False
Name: 가수, Length: 100, dtype: bool

In [48]:
condition = song_df['가수'] == '방탄소년단'
song_df.loc[condition, ['곡명', '앨범', '가사']]

,곡명,앨범,가사
2,Dynamite,Dynamite (DayTime Version),Cos ah ahI’m in the stars tonightSo watch me b...
8,Life Goes On,BE,어느 날 세상이 멈췄어아무런 예고도 하나 없이봄은 기다림을 몰라서눈치 없이 와버렸어...
38,작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey),MAP OF THE SOUL : PERSONA,모든 게 궁금해How’s your dayOh tell me뭐가 널 행복하게 하는지O...
50,봄날,YOU NEVER WALK ALONE,보고 싶다이렇게 말하니까 더 보고 싶다너희 사진을 보고 있어도보고 싶다너무 야속한 ...
93,ON,MAP OF THE SOUL : 7,I can't understand what people are sayin'어느 장단...
95,Blue & Grey,BE,Where is my angel하루의 끝을 드리운Someone come and sa...
97,내 방을 여행하는 법,BE,떠나볼까 let me fly to my시선을 낮추고 어디든 막 zoom지금 나와 l...


In [51]:
# 아이유의 노래와 앨범, 장르를 가져오세요
condition = song_df['가수'] == '아이유'
song_df.loc[condition, ['곡명', '앨범', '장르']]




,곡명,앨범,장르
16,에잇(Prod.&Feat. SUGA of BTS),에잇,록/메탈
30,Blueming,Love poem,록/메탈
36,마음을 드려요,사랑의 불시착 OST Part 11,"발라드, 국내드라마"
74,미리 메리 크리스마스 (Feat. 천둥 Of MBLAQ),Real,댄스
76,Love poem,Love poem,록/메탈


In [65]:
# 장르가 댄스인 곡명부터 장르까지 가져오세요

song_df.loc[song_df['장르'] == '댄스', '곡명':'장르'].sort_values(by='발매일', ascending=False).reset_index(drop=True)

,곡명,가수,앨범,발매일,장르
0,나로 바꾸자 (duet with JYP),비,나로 바꾸자 (duet with JYP),2020.12.31,댄스
1,Panorama,IZ*ONE (아이즈원),One-reeler / Act IV,2020.12.07,댄스
2,Black Mamba,aespa,Black Mamba,2020.11.17,댄스
3,I CAN’T STOP ME,TWICE (트와이스),Eyes wide open,2020.10.26,댄스
4,딩가딩가 (Dingga),마마무 (Mamamoo),딩가딩가 (Dingga),2020.10.20,댄스
5,DON'T TOUCH ME,환불원정대,DON'T TOUCH ME,2020.10.10,댄스
6,Lovesick Girls,BLACKPINK,THE ALBUM,2020.10.02,댄스
7,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스
8,When We Disco (Duet with 선미),박진영,When We Disco,2020.08.12,댄스
9,다시 여기 바닷가,"싹쓰리 (유두래곤, 린다G, 비룡)",다시 여기 바닷가,2020.07.18,댄스
